# Watson Machine Learing Python Function登録サンプル

## Step 1 Watson MLのWebサービス呼出し

In [1]:
# wml認証情報 (Watson Machine Learningのcredntailsをコピペします)

wml_credentials = {
}

In [2]:
# Machine LearningスコアリングURL (WebサービスエンドポイントのScoring End-pointをコピペします)
scoring_url = ''

In [3]:
# Webサービス呼出しテスト用データ
payload = {"fields": ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", 
  "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], 
"values": 
[[0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98],
[0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14]]}

In [4]:
# Webサービス呼出しのテスト
# 普通にWebサービス呼出しができることを確認します。

from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
client.deployments.score(scoring_url, payload)

{'fields': ['$E-PRICE'],
 'values': [[30.003843377020388], [25.025562379060613]]}

## Step 2 Python Functionの定義

In [5]:
# Python関数呼び出し時に必要となる情報を ai_parmsという変数にまとめておきます
ai_parms = { "wml_credentials" : wml_credentials,
             "model_deployment_endpoint_url" : scoring_url }

In [6]:
# Python Functionの定義
# 上でテストしたWebサービス呼出しをPython Functionとして定義します。

def spss_function(parms=ai_parms):
    from watson_machine_learning_client import WatsonMachineLearningAPIClient
    client = WatsonMachineLearningAPIClient( parms["wml_credentials"] )
    def score(payload):
        return client.deployments.score(parms["model_deployment_endpoint_url"], payload)
    return score

In [7]:
# Python Functionのテスト
# 上で定義したPython Functionをテストします。
function_result = spss_function()(payload)
print(function_result)

{'fields': ['$E-PRICE'], 'values': [[30.003843377020388], [25.025562379060613]]}


## Step 3 Python FunctionをWatson MLに登録

In [8]:
# PythonFunctionをstore_function関数で登録します

meta_data = { client.repository.FunctionMetaNames.NAME : 'SPSS boston regression' }
function_details = client.repository.store_function( meta_props=meta_data, function=spss_function )

No matching default runtime found. Creating one...SUCCESS

Successfully created runtime with uid: da7caab5-d7d3-4c6c-b601-05daa0044eec


In [9]:
# Function IDを取得します

function_id = function_details["metadata"]["guid"]
print(function_id)

92cb9365-08fb-4d9a-827c-536fe51ad4ea


In [10]:
# Functionをdeployします

function_deployment_details = client.deployments.create( artifact_uid=function_id, name='SPSS boston deployment' )



#######################################################################################

Synchronous deployment creation for uid: '92cb9365-08fb-4d9a-827c-536fe51ad4ea' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4ace00d9-f5af-40d1-a960-65a82ebc7ec0'
------------------------------------------------------------------------------------------------




In [11]:
# functionのdeployment URLを取得します

function_deployment_endpoint_url = client.deployments.get_scoring_url( function_deployment_details )
print(function_deployment_endpoint_url )

https://us-south.ml.cloud.ibm.com/v3/wml_instances/fa53a255-4a53-4546-8208-9b92894ab3ba/deployments/4ace00d9-f5af-40d1-a960-65a82ebc7ec0/online


In [12]:
# deploy したPython Functionの呼出しテスト

client.deployments.score( function_deployment_endpoint_url, payload)

{'fields': ['$E-PRICE'],
 'values': [[30.003843377020388], [25.025562379060613]]}